In [2]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=394):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "071970"
    company_name = "HD현대마린엔진"

    # 데이터 수집
    df = get_day_price(code, max_pages=394)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가  전일비      시가      고가      저가     거래량
2025.04.25 36750.0 2050 35650.0 37200.0 35050.0 1000466
2025.04.24 34700.0  900 33800.0 35900.0 33300.0 1036676
2025.04.23 33800.0  250 33750.0 33850.0 33000.0  339727
2025.04.22 33550.0  500 32300.0 33900.0 32300.0  475112
2025.04.21 33050.0 -900 33500.0 33950.0 32200.0  721969
2025.04.18 33950.0 -400 34650.0 34800.0 33150.0  496130
2025.04.17 34350.0  450 33800.0 34800.0 33500.0  567932
2025.04.16 33900.0    0 34100.0 34600.0 32950.0  612943
2025.04.15 33900.0    0 34850.0 35000.0 33300.0  895779
2025.04.14 33900.0 2700 32700.0 34000.0 32350.0 2184243
2025.04.11 31200.0  3050 28650.0 32050.0 28400.0 2609697
2025.04.10 28150.0  2000 27850.0 28300.0 27100.0  519019
2025.04.09 26150.0  -550 27500.0 27550.0 25500.0  514494
2025.04.08 26700.0  1800 26100.0 26950.0 25550.0  380892
2025.04.07 24900.0 -3000 26500.0 26800.0 24750.0  478374
2025.04.04 27900.0   -50 27450.0 28900.0 26900.0  409153
2025.04.03 27950.0    50 27050.0 28100.0 2